# Demo of In-Memory & Distributed Caches

In [ ]:
import os
from dotenv import load_dotenv
from src.kv_cache import KVGraphCache

load_dotenv()

config = {'db_name': os.getenv("DB_NAME"),
          'db_username': os.getenv("DB_USERNAME"),
          'db_password': os.getenv("DB_PASSWORD"),
          'db_system': os.getenv("DB_SYSTEM"),
          'db_config_file_path': os.getenv("DB_CONFIG_PATH"),
          'db_queries_file_path': os.getenv("DB_QUERIES_PATH"),
          }

in_memory_cache = KVGraphCache(config=config)

### Store Json Compatible data

In [ ]:
# store any json compatible structure
#
in_memory_cache.set_kv(store_name='my_data', key='rec_1', value={'list_data': [1,2,3], 'dict_data':{'a': 1, 'b': {'c': 2}}})

in_memory_cache.set_kv(store_name='my_data', key='rec_2', value={'dict_data':{'a': 1, 'b': {'c': 2}}})

In [ ]:
# retrieve form the cache
#
record = in_memory_cache.get_kv(store_name='my_data', key='rec_1')
print(record)

### Persist to ArangoDB

In [ ]:
# persist the data to ArangoDB
#
in_memory_cache.persist()

### Delete from cache

In [ ]:
in_memory_cache.del_kv(store_name='my_data')

In [ ]:
# persist the data to ArangoDB
#
in_memory_cache.persist()

### Store Graphs

In [ ]:
# Store AMFGraphs
#
from src.amgraph import AMFGraph
g = AMFGraph()

g.set_edge(source=('Trade', 'XYZ_123'), target=('client','abc ltd'), edge=('has', 'client'), prob=1.0)

print(g)

in_memory_cache.set_kv(store_name='my_data', key='rec_3', value=g)

In [ ]:
print(in_memory_cache.get_kv(store_name='my_data', key='rec_3'))

### Persist Graphs

In [ ]:
in_memory_cache.persist()


### Restore cache from ArangoDB

In [ ]:
in_memory_cache_2 = KVGraphCache(config=config)

in_memory_cache_2.restore(store_name='my_data', key='rec_3')

In [ ]:
restored_graph = in_memory_cache_2.get_kv(store_name='my_data', key='rec_3')
print(restored_graph)


In [ ]:
restored_graph.update_edge(source=('Trade', 'XYZ_123'), target=('client','abc ltd'), edge=('has', 'client'), prob=0.5)

print(restored_graph)

In [ ]:
in_memory_cache_2.set_kv(store_name='my_data', key='rec_3', value=restored_graph)

In [ ]:
in_memory_cache_2.persist()


### Distribute a Cache Using Dask

In [ ]:
from src.distributed_cache import DistributedCache

config['scheduler_address']= 'tcp://192.168.1.67:8786'

dc = DistributedCache(config=config)

In [ ]:
dc.create_distributed_store(store_name='my_data', key='rec_3', restore=True)

### Get out the persisted graph 

In [ ]:
future = dc.get_kv(store_name='my_data', key='rec_3')

future

In [ ]:
remote_graph = future.result()
print(remote_graph)

In [ ]:
remote_graph.update_edge(source=('Trade', 'XYZ_123'), target=('client','abc ltd'), edge=('has', 'client'), prob=0.75)

print(remote_graph)

In [ ]:
future = dc.set_kv(store_name='my_data', key='rec_3', value=remote_graph)
future

In [ ]:
future.result()